In [1]:
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Conv2DTranspose, concatenate
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.losses import SparseCategoricalCrossentropy


ImportError: Traceback (most recent call last):
  File "C:\Users\Yahnis\.conda\envs\env_ia\lib\site-packages\tensorflow\python\pywrap_tensorflow.py", line 62, in <module>
    from tensorflow.python._pywrap_tensorflow_internal import *
ImportError: DLL load failed while importing _pywrap_tensorflow_internal: Une routine d’initialisation d’une bibliothèque de liens dynamiques (DLL) a échoué.


Failed to load the native TensorFlow runtime.
See https://www.tensorflow.org/install/errors for some common causes and solutions.
If you need help, create an issue at https://github.com/tensorflow/tensorflow/issues and include the entire stack trace above this error message.

In [2]:
def convert_rgb_to_labels(mask_img, color_to_label):
    label_mask = np.zeros((mask_img.shape[0], mask_img.shape[1]), dtype=np.int32)
    for rgb, label in color_to_label.items():
        mask = np.all(mask_img == np.array(rgb, dtype=np.uint8), axis=-1)
        label_mask[mask] = label
    return label_mask

def load_images_and_masks(satellite_folder, mask_folder):
    satellite_images = []
    masks = []
    for filename in os.listdir(satellite_folder):
        sat_img = Image.open(os.path.join(satellite_folder, filename))
        mask_img = Image.open(os.path.join(mask_folder, filename))  # Assurez-vous que les noms correspondent

        satellite_images.append(np.array(sat_img))
        converted_mask = convert_rgb_to_labels(np.array(mask_img), color_to_label)
        masks.append(converted_mask)

    return np.array(satellite_images), np.array(masks)


In [3]:
color_to_label = {
    (180, 180, 180): 0,  # Gris - Végétation clairsemée
    (0, 100, 0): 1,     # Vert - Couverture arborée
    (255, 187, 34): 2,  # Orange - Broussailles
    (255, 255, 76): 3,  # Jaune - Prairie
    (240, 150, 255): 4, # Rose - Terre cultivée
    (250, 0, 0): 5,     # Rouge - Zones urbanisées
    (240, 240, 240): 6, # Blanc - Neige et glace
    (0, 100, 200): 7,   # Bleu - Plans d'eau permanents
    (0, 150, 160): 8,   # Vert-bleu - Marais herbacé
    (0, 207, 117): 9,   # Vert mangrove - Mangroves
    (250, 230, 160): 10 # Mousse - Lichens
}

In [4]:
path_satellite_images = 'TrainData\Esa_tile1_Sat'  # Chemin vers les images satellites
path_labelled_images = 'TrainData\Esa_tile1_resize'  # Chemin vers les images labelisées (masques)

X, Y = load_images_and_masks(path_satellite_images, path_labelled_images)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)


C:\Users\adrie\AppData\Local\Temp\ipykernel_12544\3415542517.py:4: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  mask = np.all(mask_img == np.array(rgb, dtype=np.uint8), axis=-1)


In [5]:
def create_unet_model(input_shape, num_classes):
    inputs = Input(input_shape)

    # Couches descendantes
    c1 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(inputs)
    p1 = MaxPooling2D((2, 2))(c1)
    
    c2 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p1)
    p2 = MaxPooling2D((2, 2))(c2)
    
    c3 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p2)
    p3 = MaxPooling2D((2, 2))(c3)
    
    c4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p3)
    p4 = MaxPooling2D(pool_size=(2, 2))(c4)
    
    c5 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p4)

    # Couches ascendantes
    u6 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
    u6 = concatenate([u6, c4])
    c6 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u6)
    
    u7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = concatenate([u7, c3])
    c7 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u7)
    
    u8 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c7)
    u8 = concatenate([u8, c2])
    c8 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u8)
    
    u9 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c8)
    u9 = concatenate([u9, c1], axis=3)
    c9 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u9)

    outputs = Conv2D(num_classes, (1, 1), activation='softmax')(c9)

    model = Model(inputs=[inputs], outputs=[outputs])
    return model

# Remplacer par la forme réelle de vos images et le nombre de canaux (par exemple, (1440, 1440, 3) pour des images RGB)
model = create_unet_model((1440, 1440, 3), num_classes=11)
model.compile(optimizer='adam', 
              loss=SparseCategoricalCrossentropy(from_logits=False),  # Assuming softmax activation in the last layer
              metrics=['accuracy'])

In [6]:
model.fit(X_train, Y_train, batch_size=4, epochs=1, validation_data=(X_test, Y_test))
model.evaluate(X_test, Y_test)

2/2 [==============================] - 38s 2s/step - loss: 8.8523e-06 - accuracy: 1.0000


[8.852269274939317e-06, 0.9999993443489075]

In [11]:
from PIL import Image, ImageOps
import numpy as np
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.metrics import SparseCategoricalCrossentropy

# Charger et prétraiter l'image
def load_and_preprocess_image(image_path, target_size):
    image = load_img(image_path, target_size=target_size) # Assurez-vous que target_size correspond à la taille d'entrée du modèle
    image = img_to_array(image)
    image = np.expand_dims(image, axis=0) # Ajouter une dimension pour le batch
    return image

# Convertir une prédiction en image PIL
def prediction_to_image(prediction):
    prediction = np.argmax(prediction, axis=-1) # Prendre l'indice de la classe la plus probable
    prediction = np.uint8(prediction * 255 / prediction.max()) # Convertir en valeurs d'octets pour l'affichage
    return Image.fromarray(prediction)

# Faire une prédiction et afficher avec Pillow
def predict_and_display(model, image_path, target_size=(1440, 1440)):
    image = load_and_preprocess_image(image_path, target_size)
    prediction = model.predict(image)[0] # Récupérer la prédiction pour le premier élément du batch

    # Convertir la prédiction en image PIL et afficher
    original_image = Image.open(image_path)
    predicted_image = prediction_to_image(prediction)

    original_image.show(title='Image Originale')
    predicted_image.show(title='Prédiction du Modèle')

# Exemple d'utilisation
image_path = 'TrainData/Esa_tile1_Sat/ESA_WorldCover_10m_2021_V200_N33W006_Map.tif' # Remplacer par le chemin de votre image
predict_and_display(model, image_path)

1/1 [==============================] - 1s 733ms/step


TypeError: Cannot handle this data type: (1, 1, 1440), |u1